# 🐎 Horse Detection using Computer Vision with YOLOv9 🐎

Welcome to our notebook on `Horse Detection using Computer Vision with YOLOv9!` In this notebook, we will walk you through the process of training a custom YOLOv9 model to detect horses in images and videos. 🐎

![image](https://wallpapers.com/images/featured/horse-h3azzzaaorg8c9ay.jpg)

## Overview

- **Data Collection**: We will download a dataset containing images and videos of horses from Roboflow.
- **Model Training**: Using YOLOv9, we will train a custom model on the dataset to accurately detect horses.
- **Model Evaluation**: We will evaluate the trained model's performance using validation images and videos.
- **Inference**: Finally, we will perform inference on new images and videos to detect horses in real-world scenarios.

Let's dive in and explore the exciting world of horse detection with computer vision! 🌟

#### **Note:** We will display one original video with its detected one that we downloaded after running this notebook before sharing it on Kaggle

In [ ]:
import glob
from IPython.display import HTML
from base64 import b64encode

def play_video(filename, video_title):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += f'<div style="text-align:center;">'
    html += f'<h3>{video_title}</h3>'
    html += '<video width=800 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [ ]:
play_video('/kaggle/input/horses-test-videos/Horses_1.mp4', "Original Video")

In [ ]:
play_video('/kaggle/input/detected-horses-videos/Detected_Horses_1.mp4', "Detected Video")

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Settings` -> `Accelerator`, and then set it to `GPU`.

In [ ]:
!nvidia-smi

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Clone and Install

**NOTE:** YOLOv9 is very new. At the moment, we recommend using a fork of the main repository. The `detect.py` script contains a bug that prevents inference. This bug is patched in the fork.

In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

**NOTE:** Let's install the [`roboflow`](https://pypi.org/project/roboflow) package, which we will use to download our dataset from [Roboflow Universe](https://universe.roboflow.com/).

In [ ]:
!pip install roboflow -q

## Download model weights

**NOTE:** In the YOLOv9 paper, versions `yolov9-s` and `yolov9-m` are also mentioned, but the weights for these models are not yet available in the YOLOv9 [repository](https://github.com/WongKinYiu/yolov9).

In [ ]:
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P {HOME}/weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!ls -la {HOME}/weights

## Download the Dataset

**NOTE:** The dataset must be saved inside the `{HOME}/yolov9` directory, otherwise, the training will not succeed.

In [ ]:
%cd {HOME}/yolov9

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="G2zfOgnsvFlAUDu2L1Vy")
project = rf.workspace("saban-ne0tf").project("horse-jehyp")
version = project.version(1)
dataset = version.download("yolov9")

### Display Some Random Training Images

In [ ]:
import random
import matplotlib.pyplot as plt
from PIL import Image

# Path to the images folder
images_folder = '/kaggle/working/yolov9/horse-1/train/images'

# Function to load and display images
def display_images(image_paths):
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    for i, image_path in enumerate(image_paths):
        image = Image.open(image_path)
        axes[i].imshow(image)
        axes[i].axis('off')
    plt.show()

# Get a list of all image paths in the training folder
image_paths = [os.path.join(images_folder, filename) for filename in os.listdir(images_folder) if filename.endswith('.jpg')]

# Shuffle the list of image paths
random.shuffle(image_paths)

# Select 8 random image paths
random_image_paths = image_paths[:8]

# Display the random images
display_images(random_image_paths)

## Train Custom Model

In [ ]:
%cd {HOME}/yolov9

!python train.py \
--batch 16 --epochs 30 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data {dataset.location}/data.yaml \
--weights {HOME}/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

## Examine Training Results

**NOTE:** By default, the results of each subsequent training sessions are saved in `{HOME}/yolov9/runs/train/`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
!ls {HOME}/yolov9/runs/train/exp/

In [ ]:
from IPython.display import Image

Image(filename=f"{HOME}/yolov9/runs/train/exp/results.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp/confusion_matrix.png", width=1000)

In [ ]:
Image(filename=f"{HOME}/yolov9/runs/train/exp/val_batch0_pred.jpg", width=1000)

## Validate Custom Model

In [ ]:
%cd {HOME}/yolov9

!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data {dataset.location}/data.yaml \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt

## Inference with Custom Model on some images and videos

In [ ]:
# Detect some test images
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source /kaggle/input/horse-test-images

In [ ]:
import glob

# Display some test images
for image_path in glob.glob(f'{HOME}/yolov9/runs/detect/exp/*.jpg'):
      display(Image(filename=image_path, width=600))

In [ ]:
# Detect some test videos
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights {HOME}/yolov9/runs/train/exp/weights/best.pt \
--source /kaggle/input/horses-test-videos

#### You can download the detected videos from output environment and the following is an example one of the detected videos like we displayed at the start of the notebook

In [ ]:
play_video('/kaggle/input/detected-horses-videos/Detected_Horses_9.mp4', "Detected Video")

# How to Train YOLOv9 on a Custom Dataset
---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/train-yolov9-model/)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/WongKinYiu/yolov9)
[![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/XHT2c8jT3Bc)
[![arXiv](https://img.shields.io/badge/arXiv-2402.13616-b31b1b.svg)](https://arxiv.org/pdf/2402.13616.pdf)

# Made by: Abdelrahman Eldaba 👨‍💻

Check out my website with a portfolio [Here](https://sites.google.com/view/abdelrahman-eldaba110) 🌟

Connect with me on [LinkedIn](https://www.linkedin.com/in/abdelrahmaneldaba/) 🌐

Look at my [GitHub](https://github.com/Abdelrahman47-code) 🚀